<a href="https://colab.research.google.com/github/01SakuraAyane29/Machine-Learning-Models/blob/master/Regressors.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import scipy as sp
import sklearn as sk

In [0]:
from google.colab import files
uploaded = files.upload()
for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

Saving kc_house_data.csv to kc_house_data.csv
User uploaded file "kc_house_data.csv" with length 2515206 bytes


In [0]:
import time
import datetime

data = pd.read_csv('kc_house_data.csv',sep=',')
y_data = data['price']
x_data = data.drop(columns = ['id','date','price'])

date = data['date']

date = date.apply(lambda x: time.strptime(x[2:4]+x[4:6]+x[6:8], '%y%m%d'))

date = date.apply(lambda x: (time.mktime(datetime.datetime.now().timetuple())
                             -time.mktime(x))/86400)

x_data.insert(1,'date',date)


In [0]:
from sklearn.model_selection import train_test_split

x_data.dropna(axis = 0)
y_data.dropna(axis = 0)

x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size=0.33, random_state=42)

In [0]:
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import SGDRegressor
from sklearn.preprocessing import MaxAbsScaler

my_model = SGDRegressor()

scaler = MaxAbsScaler()

sc_x_train = x_train.copy()
sc_y_train = y_train.copy().values.reshape(-1,1)

sc_x_train = scaler.fit_transform(sc_x_train)
sc_y_train = scaler.fit_transform(sc_y_train)

my_model.fit(sc_x_train,sc_y_train.ravel())
scores = cross_val_score(estimator = my_model, X = sc_x_train,
                                      y = sc_y_train, cv = 5, n_jobs = -1,scoring = 'neg_mean_absolute_error')
print('Cross validation scores: %s' % scores)
print('Cross validation : %.4f +/- %.4f' % (np.mean(scores),np.std(scores)))
print(scaler.inverse_transform(np.mean(scores)))

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDRegressor'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)

Cross validation scores: [-0.01948598 -0.01950306 -0.02034836 -0.01986191 -0.01965334]
Cross validation : -0.0198 +/- 0.0003
[[-152233.07377931]]


In [0]:
from sklearn.feature_selection import *
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import MaxAbsScaler
from sklearn.linear_model import SGDRegressor

my_model = SGDRegressor(penalty = 'elasticnet')

scaler = MaxAbsScaler()


training_scores = []
validation_scores = []
scores = []
abs_validation_scores = []

selection_x = x_train.copy()
selection_y = y_train.copy().values.reshape(-1,1)

selection_x = scaler.fit_transform(selection_x)
selection_y = scaler.fit_transform(selection_y)


for selector in [SelectKBest(f_regression, k = 5),SelectKBest(f_regression),
                 SelectPercentile(f_regression),SelectFpr(f_regression),SelectFdr(f_regression)]:
  loop_x = selection_x.copy()
  loop_y = selection_y.copy()
  loop_x = selector.fit_transform(loop_x, loop_y)
  my_model.fit(loop_x,loop_y)
  training_scores.append(my_model.score(loop_x, loop_y))
  abs_validation_scores = np.asarray(cross_val_score(estimator = my_model, X = loop_x, y = loop_y, cv = 5, n_jobs = -1,scoring = 'neg_mean_absolute_error'))
  abs_mean = -np.mean(abs_validation_scores)
  abs_mean = np.asarray(abs_mean)
  scores.append(scaler.inverse_transform(abs_mean.reshape(-1,1)))
  abs_validation_scores =[]

print(training_scores)

print(scores)

/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDRegressor'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)

[0.4648091160444773, 0.5341956713503366, 0.38411921230293117, 0.5569830050655015, 0.556684886621069]
[array([[166352.9265979]]), array([[156845.67496814]]), array([[179582.28763901]]), array([[153806.51309795]]), array([[153669.26096306]])]


In [0]:
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import MaxAbsScaler

my_model = MLPRegressor(hidden_layer_sizes = 1000)

scaler = MaxAbsScaler()

sc_x_train = x_train.copy()
sc_y_train = y_train.copy().values.reshape(-1,1)

sc_x_train = scaler.fit_transform(sc_x_train)
sc_y_train = scaler.fit_transform(sc_y_train)

my_model.fit(sc_x_train,sc_y_train.ravel())
scores = cross_val_score(estimator = my_model, X = sc_x_train,
                                      y = sc_y_train, cv = 5, n_jobs = -1,scoring = 'neg_mean_absolute_error')
print('Cross validation scores: %s' % scores)
print('Cross validation : %.4f +/- %.4f' % (np.mean(scores),np.std(scores)))
print(scaler.inverse_transform(np.mean(scores)))


/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/multilayer_perceptron.py:1306: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/neural_network/multilayer_perceptron.py:1306: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Cross validation scores: [-0.01733623 -0.01751626 -0.01789378 -0.01832199 -0.01780026]
Cross validation : -0.0178 +/- 0.0003
[[-136857.51705642]]


In [0]:
df_usa = pd.read_csv("kc_house_data.csv", sep=',')

In [0]:
df_usa = df_usa.merge(pd.get_dummies(df_usa.floors, drop_first=True, prefix='Floors'), left_index=True, right_index=True)
df_usa = df_usa.merge(pd.get_dummies(df_usa.waterfront, drop_first=True, prefix='watFront'), left_index=True, right_index=True)
df_usa = df_usa.merge(pd.get_dummies(df_usa.view, drop_first=True, prefix='View'), left_index=True, right_index=True)
df_usa = df_usa.merge(pd.get_dummies(df_usa.condition, drop_first=True, prefix='Cond'), left_index=True, right_index=True)
df_usa = df_usa.merge(pd.get_dummies(df_usa.grade, prefix='Grade'), left_index=True, right_index=True)
df_usa = df_usa.merge(pd.get_dummies(df_usa.bedrooms, drop_first=True, prefix='Bedrooms'), left_index=True, right_index=True)

del df_usa['floors'],
del df_usa['waterfront']
del df_usa['view']
del df_usa['condition']
del df_usa['grade']
del df_usa['bedrooms']

In [0]:
from sklearn.model_selection import train_test_split
df_usa.drop(['id', 'date'], axis=1, inplace=True)
X = df_usa.drop('price',axis=1).values
y = df_usa['price'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state=3)

In [0]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import RandomizedSearchCV
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 10, stop = 200, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

# Use the random grid to search for best hyperparameters
# First create the base model to tune
rf = RandomForestRegressor()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 100, cv = 3, verbose=2, random_state=42, n_jobs = -1)
# Fit the random search model
rf_random.fit(X_train, y_train)

Fitting 3 folds for each of 100 candidates, totalling 300 fits
[CV] n_estimators=31, min_samples_split=5, min_samples_leaf=1, max_features=sqrt, max_depth=30, bootstrap=True 
[CV] n_estimators=31, min_samples_split=5, min_samples_leaf=1, max_features=sqrt, max_depth=30, bootstrap=True 
[CV]  n_estimators=31, min_samples_split=5, min_samples_leaf=1, max_features=sqrt, max_depth=30, bootstrap=True, total=   1.2s
[CV] n_estimators=31, min_samples_split=5, min_samples_leaf=1, max_features=sqrt, max_depth=30, bootstrap=True 
[CV]  n_estimators=31, min_samples_split=5, min_samples_leaf=1, max_features=sqrt, max_depth=30, bootstrap=True, total=   1.2s
[CV] n_estimators=200, min_samples_split=5, min_samples_leaf=1, max_features=sqrt, max_depth=10, bootstrap=True 
[CV]  n_estimators=31, min_samples_split=5, min_samples_leaf=1, max_features=sqrt, max_depth=30, bootstrap=True, total=   0.9s
[CV] n_estimators=200, min_samples_split=5, min_samples_leaf=1, max_features=sqrt, max_depth=10, bootstrap=

[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:  3.0min


[CV]  n_estimators=200, min_samples_split=10, min_samples_leaf=1, max_features=sqrt, max_depth=10, bootstrap=False, total=   6.7s
[CV] n_estimators=157, min_samples_split=5, min_samples_leaf=2, max_features=sqrt, max_depth=10, bootstrap=False 
[CV]  n_estimators=157, min_samples_split=5, min_samples_leaf=2, max_features=sqrt, max_depth=10, bootstrap=False, total=   5.5s
[CV] n_estimators=157, min_samples_split=5, min_samples_leaf=2, max_features=sqrt, max_depth=10, bootstrap=False 
[CV]  n_estimators=200, min_samples_split=10, min_samples_leaf=1, max_features=sqrt, max_depth=10, bootstrap=False, total=   7.0s
[CV] n_estimators=157, min_samples_split=5, min_samples_leaf=2, max_features=sqrt, max_depth=10, bootstrap=False 
[CV]  n_estimators=157, min_samples_split=5, min_samples_leaf=2, max_features=sqrt, max_depth=10, bootstrap=False, total=   5.4s
[CV] n_estimators=73, min_samples_split=10, min_samples_leaf=2, max_features=sqrt, max_depth=30, bootstrap=False 
[CV]  n_estimators=157, mi

[Parallel(n_jobs=-1)]: Done 158 tasks      | elapsed: 13.9min


[CV]  n_estimators=94, min_samples_split=10, min_samples_leaf=4, max_features=auto, max_depth=50, bootstrap=False, total=  16.9s
[CV] n_estimators=94, min_samples_split=5, min_samples_leaf=4, max_features=auto, max_depth=30, bootstrap=False 
[CV]  n_estimators=94, min_samples_split=5, min_samples_leaf=4, max_features=auto, max_depth=30, bootstrap=False, total=  17.1s
[CV] n_estimators=94, min_samples_split=5, min_samples_leaf=4, max_features=auto, max_depth=30, bootstrap=False 
[CV]  n_estimators=94, min_samples_split=5, min_samples_leaf=4, max_features=auto, max_depth=30, bootstrap=False, total=  17.0s
[CV] n_estimators=136, min_samples_split=5, min_samples_leaf=2, max_features=sqrt, max_depth=None, bootstrap=False 
[CV]  n_estimators=94, min_samples_split=5, min_samples_leaf=4, max_features=auto, max_depth=30, bootstrap=False, total=  17.0s
[CV] n_estimators=136, min_samples_split=5, min_samples_leaf=2, max_features=sqrt, max_depth=None, bootstrap=False 
[CV]  n_estimators=136, min_s

[Parallel(n_jobs=-1)]: Done 300 out of 300 | elapsed: 25.8min finished


RandomizedSearchCV(cv=3, error_score='raise',
          estimator=RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=False),
          fit_params=None, iid=True, n_iter=100, n_jobs=-1,
          param_distributions={'n_estimators': [10, 31, 52, 73, 94, 115, 136, 157, 178, 200], 'max_features': ['auto', 'sqrt'], 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, None], 'min_samples_split': [2, 5, 10], 'min_samples_leaf': [1, 2, 4], 'bootstrap': [True, False]},
          pre_dispatch='2*n_jobs', random_state=42, refit=True,
          return_train_score='warn', scoring=None, verbose=2)

In [0]:
from sklearn.metrics import mean_absolute_error
print(rf_random.best_params_)
y_hat_Search = rf_random.predict(X_test)
print("MSE of RandomForest Regressor: {} ".format(mean_absolute_error(y_hat_Search, y_test)))

{'n_estimators': 157, 'min_samples_split': 5, 'min_samples_leaf': 1, 'max_features': 'auto', 'max_depth': 90, 'bootstrap': True}
MSE of RandomForest Regressor: 68695.22171451294 
